# Question 1

What tests have we learned, and when should they be applied (on what kind of data)? Can you also specify when a test needs special ways to deal with ties.

## Paired data $\{(x_i, y_i) \; | \; i = 1, \ldots, n\}$
  - Same location parameter? Is the dist for $D = Y-X$ centered on zero? I.e., test on $d_i = y_i - x_i$. 
    - Sign test
    - (Wilcoxon) Sign rank test
  - Correlation: Spreaman $\rho$, Kendal $\tau$

## Independent samples $\{x_i\; |\; i=1, \ldots , n\}, \{y_j\; |\; j=1, \ldots , m\}$
  - Equal variances? Conover sqaured ranks test.
  - Same location parameter? Count number of times $x_i > y_j$ for all $i$ and $j$. Mann-Whitney test $\equiv$ (Wilcoxon) rank sum test.
  - Compare EDFs: Kolmogorov-Smirnov, Cramer-von Mises, or Anderson-Darling test

## $k$ samples: $\{x_{i,j}\;|\; i=1, \ldots,k; j=1, \ldots, n_i\}$ 
Generalization: $x_j \rightarrow x_{1,j}$, $y_j\rightarrow x_{2, j}$, $n\rightarrow n_1$, $m\rightarrow n_2$ etc.

- Rank sum test $\rightarrow$ Kruskal-Wallis test
  - Conover-Iman test to check which pairs of samples are responsible.
- 2-sample Conover squared ranks $\rightarrow$ $k$-sample squared ranks

## Complete block design: $\{x_{i,j}\;|\;i=1, \ldots, b; j=1, \ldots, k\}$
Compare "treatments", labelled by $j$ within each block labelled by $i$. Generalize paired data to $k$ "samples".

$x_i \rightarrow x_{i1}$; $y_i \rightarrow x_{i,2}$ $n\rightarrow b$

- Sign test $\rightarrow$ Friedman (rank treatments within blocks)
- Sign rank test $\rightarrow$ Quade test (weight ranks by spread within block)

## ROC curves/ARE/power curves; one sample K-S etc. tests

# Question 2

For $p$-values: use Monte Carlo or built-in functions?

Answer: Generally okay to use functions providing null distribution (e.g. `stats.ksone()` for Kolmogorov distribution), but don't use function to do the whole test for you.

# Question 3

One-sample K-S test for discrete data; what replaces Kolomogorov distribution?

Method in Conover section 6.1 (**A Method of Obtain the Exact $p$-Value When $F^*(x)$ is Discrete**) e.g., Example 2, is not recommended.

Better to do a Monte Carlo as in lesson 7.2.

# Review

...

In [1]:
%matplotlib inline

In [2]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import itertools
plt.rcParams['figure.figsize'] = (8.0,5.0)
plt.rcParams['font.size'] = 14

## Kruskal-Wallis

The null is assumed Chi squared.

In [3]:
def kruskal_wallis(x_i_j):
    n_i = np.array([len(xi_j) for xi_j in x_i_j])
    k = len(n_i)
    N = np.sum(n_i)
    
    x_r = np.concatenate(x_i_j)
    R_r = stats.rankdata(x_r)
    i_r = np.concatenate([(i,)*n_i[i] for i in range(k)])
    R_i_j = [R_r[i_r==i] for i in range(k)]
    R_i = np.array([np.sum(Ri_j) for Ri_j in R_i_j])
    
    Rbar = 0.5*(N+1)
    T = (N-1) * np.sum((R_i-n_i*Rbar)**2/n_i) / np.sum((R_r-Rbar)**2)
    
    p = stats.chi2(df=k-1).sf(T)
    
    return T, p

In [4]:
x_i_j = [
    np.array([ 14.97,   5.80,  25.03,   5.50 ]),
    np.array([  5.83,  13.96,  21.96]),
    np.array([ 17.89,  23.03,  61.09,   18.62,  55.51])
]
kruskal_wallis(x_i_j)

(4.153846153846154, 0.12531520484413722)

## Conover-Iman

The null is `t`.

In [5]:
def conover_iman(newx_i_j):
    n_i = np.array([len(xi_j) for xi_j in newx_i_j])
    k = len(n_i)
    N = np.sum(n_i)
    
    x_r = np.concatenate(newx_i_j)
    R_r = stats.rankdata(x_r)
    i_r = np.concatenate([(i,)*n_i[i] for i in range(k)])
    R_i_j = [R_r[i_r==i] for i in range(k)]
    R_i = np.array([np.sum(Ri_j) for Ri_j in R_i_j])
    
    Rbar = 0.5*(N+1)
    T = (N-1) * np.sum((R_i-n_i*Rbar)**2/n_i) / np.sum((R_r-Rbar)**2)
    
    newx_r = np.concatenate(newx_i_j)
    newR_r = stats.rankdata(newx_r)
    
    newR_i_j = [newR_r[i_r==i] for i in range(k)]
    
    newR_i = np.array([np.sum(Ri_j) for Ri_j in newR_i_j])
    newSsq = np.sum((newR_r-Rbar)**2)/(N-1)
    newRbar_i = newR_i/n_i
            
    newT = (N-1) * np.sum((newR_i-n_i*Rbar)**2/n_i) / np.sum((newR_r-Rbar)**2); newT
    p = stats.chi2(df=k-1).sf(newT)
    
    newT_ii = (newRbar_i[:,None]-newRbar_i[None,:])/np.sqrt(newSsq*(N-1-T)/(N-k)*(1/n_i[:,None]+1/n_i[None,:]))
    ps = 2*stats.t(df=N-k).sf(np.abs(newT_ii))
    
    return (newT, p), (newT_ii, ps)

In [6]:
newx_i_j = [
    np.array([ 14.97,   5.80,  15.03,   5.50 ]),
    np.array([  5.83,  13.96,  21.96 ]),
    np.array([ 27.89,  23.03,  61.09,   18.62,  55.51 ])
]
conover_iman(newx_i_j)

((7.476923076923077, 0.023790676031139445),
 (array([[ 0.        , -0.87060544, -4.16315718],
         [ 0.87060544,  0.        , -2.91360309],
         [ 4.16315718,  2.91360309,  0.        ]]),
  array([[1.        , 0.40659156, 0.00243626],
         [0.40659156, 1.        , 0.01721003],
         [0.00243626, 0.01721003, 1.        ]])))

## Conover Squared-Ranks

...

In [7]:
def conover_squared_ranks_k_sample(x_i_j):
    n_i = np.array([len(xi_j) for xi_j in x_i_j])
    k = len(n_i)
    N = np.sum(n_i)

    xbar_i = np.array([np.mean(xi_j) for xi_j in x_i_j])
    
    U_i_j = [np.abs(xi_j-np.mean(xi_j)) for xi_j in x_i_j]
    
    U_r = np.concatenate(U_i_j) 
    RU_r = stats.rankdata(U_r)
    i_r = np.concatenate([(i,)*n_i[i] for i in range(k)])
    RU_i_j = [RU_r[i_r==i] for i in range(k)]
    
    S_i = np.array([np.sum(RUi_j**2) for RUi_j in RU_i_j])
    Sbar = np.mean(RU_r**2)
    
    Dsq = N/(N-1)*np.mean((RU_r**2-Sbar)**2)
    T = np.sum((S_i-n_i*Sbar)**2/n_i)/Dsq
    
    p = stats.chi2(df=k-1).sf(T)
    
    return T, p

In [8]:
x_i_j = [
    np.array([ 14.97,   5.80,  25.03,   5.50 ]),
    np.array([  5.83,  13.96,  21.96]),
    np.array([ 17.89,  23.03,  61.09,   18.62,  55.51])
]
conover_squared_ranks_k_sample(x_i_j)

(7.436484543493889, 0.024276602034339175)

In [9]:
x_i = np.array([8.56, 5.03, 48.1, 1.31, 4.82]); y_j = np.array([15.0, 12.3, 28.0, 13.9])
x_i_j = [x_i, y_j]
conover_squared_ranks_k_sample(x_i_j)

(2.3133164235890935, 0.1282701146119386)

## Pearson’s $r$

...

In [10]:
def get_pearsons_r(x_i, y_i):
    xbar = np.mean(x_i)
    ybar = np.mean(y_i)
    
    return np.sum((x_i-xbar)*(y_i-ybar))/np.sqrt(np.sum((x_i-xbar)**2)*np.sum((y_i-ybar)**2))

In [11]:
x_i = np.array([9.64, 5.91, 3.22, 2.04, 5.49, 9.24, 6.38, 7.79, 0.48, 8.86])
y_i = np.array([5.53, 3.48, 3.16, 2.98, 7.11, 7.75, 3.37, 8.24, 3.00, 3.75])

get_pearsons_r(x_i, y_i)

0.5901002196595794

## Spearman’s $\rho$

In [12]:
def get_spearmans_rho(x_i, y_i):
    Rx_i = stats.rankdata(x_i)
    Ry_i = stats.rankdata(y_i)
    
    Rbar = np.mean(Rx_i)
    
    rho = np.sum((Rx_i-Rbar)*(Ry_i-Rbar) / np.sqrt(np.sum((Rx_i-Rbar)**2)*np.sum((Ry_i-Rbar)**2)))
    
    return rho

In [13]:
get_spearmans_rho(x_i, y_i)

0.7333333333333334

## Kendall’s $\tau$

In [14]:
def get_kendall_tau(x_i, y_i):
    def is_concordant(pt1,pt2):
        return ((pt1[0]>pt2[0])&(pt1[1]>pt2[1])|(pt1[0]<pt2[0])&(pt1[1]<pt2[1]))
    def is_discordant(pt1,pt2):
        return ((pt1[0]>pt2[0])&(pt1[1]<pt2[1])|(pt1[0]<pt2[0])&(pt1[1]>pt2[1]))
    
    assert len(x_i) == len(y_i)
    n = len(x_i)
    
    Nc = np.sum([is_concordant((x_i[i],y_i[i]),(x_i[j],y_i[j])) for (i,j) in itertools.combinations(range(n),2)])
    Nd = np.sum([is_discordant((x_i[i],y_i[i]),(x_i[j],y_i[j])) for (i,j) in itertools.combinations(range(n),2)])
    
    tau = (Nc-Nd)/(Nc+Nd); tau
    
    return tau

In [15]:
get_kendall_tau(x_i, y_i)

0.5555555555555556

## Friedman 

...

In [16]:
def get_friedman(X_ij):
    b, k = np.shape(X_ij)
    
    R_ij = stats.rankdata(X_ij,axis=-1)
    R_j = np.sum(R_ij,axis=0)
    
    T1 = (12/(b*k*(k+1)))*np.sum((R_j-0.5*b*(k+1))**2)
    p = stats.chi2(df=k-1).sf(T1)
    
    return T1, p

In [17]:
X_ij = np.array(
    [[  2.  ,  19.86,   9.17],
     [  1.05,   3.1 ,   3.34],
     [  0.14,  25.4 ,  26.59],
     [ 14.6 ,   3.93,  10.95]]
)
get_friedman(X_ij)

(2.0, 0.36787944117144245)

## Quade 

...

In [18]:
def quade(X_ij):
    b, k = np.shape(X_ij)
    
    M_i = np.max(X_ij,axis=-1)-np.min(X_ij,axis=-1)
    Q_i = stats.rankdata(M_i)
    
    R_ij = stats.rankdata(X_ij,axis=-1)
    R_j = np.sum(R_ij,axis=0)
    
    S_ij = Q_i[:,None]*(R_ij-0.5*(k+1))
    
    S_j = np.sum(S_ij,axis=0)
    
    B = np.sum(S_j**2)/b
    A2 = np.sum(S_ij**2)
    T3 = (b-1)*B/(A2-B)
    
    p = stats.f(k-1,(b-1)*(k-1)).sf(T3)
    
    return T3, p

In [19]:
quade(X_ij)

(1.0449438202247192, 0.40796817129629637)

## Continous Kolmagorov

...

In [20]:
def cont_kolmagorov(x_i, Fstar_i):
    n = len(x_i)
    
    Fhatp_i = np.arange(n)/n
    Fhatm_i = (1+np.arange(n))/n
    
    Tp = max(Fstar_i-Fhatp_i)
    Tm = max(Fhatm_i-Fstar_i)
    T = max(Tp, Tm)
    
    p = stats.ksone(n).sf(T)
    
    return T, p

In [21]:
x_i = np.array([-1.82, 0.72, 1.67, 1.09, 0.64, 0.81, 1.74, -0.80, -0.13, 1.12])
Fstar_i = stats.norm.cdf(np.sort(x_i))
cont_kolmagorov(x_i, Fstar_i)

(0.43891370030713844, 0.014275830316232892)

## Discrete Kolmagorov

Did not implement.

In [22]:
def dis_kolmagorov():
    pass

## Lilliefors

...

In [23]:
def lilliefors(x_i, stardist, sourcedist):
    x_i.sort()
    n = len(x_i)
    
    Fstar_i = stardist.cdf(x_i)
    Fhatp_i = np.arange(n)/n 
    Fhatm_i = (1+np.arange(n))/n
    
    Tp = max(Fstar_i-Fhatp_i) 
    Tm = max(Fhatm_i-Fstar_i)
    T = max(Tp,Tm)
    
    np.random.seed(20230327)
    Nmonte = 10**5
    x_Ii = sourcedist.rvs(size=(Nmonte,n))
    x_Ii.sort(axis=-1)
    
    xbar_I = np.mean(x_Ii,axis=-1)
    s_I = np.std(x_Ii,axis=-1,ddof=1)
    
    Fstar_Ii = sourcedist.cdf((x_Ii-xbar_I[:,None])/s_I[:,None])
    
    Tp_I = np.max(Fstar_Ii-Fhatp_i[None,:],axis=-1)
    Tm_I = np.max(Fhatm_i[None,:]-Fstar_Ii,axis=-1)
    T_I = np.maximum(Tp_I,Tm_I)
    
    p = np.mean(T_I>=T)
    
    return T, p

In [24]:
x_i = np.array([-0.6106, -0.2310, -1.0372, -2.1245, 0.7290, 0.0136, -1.4146, -1.0677, -2.6589,  0.0709, 0.7706, 3.7948, -1.4862, -0.0701, -1.3513, -0.8655, -0.2769, -0.5387, 0.2276, -0.0120, -3.8585, 0.0835, -1.7957, 1.0703, -0.6074, -0.8175, -0.9521, 0.6801, 2.5205, 0.1078, -1.2938, -0.6855, -2.1204, -0.3684, -0.4298, -1.2256, 1.3653, -2.2061, -1.6217, -2.3376,    -1.1890, -1.9026, 0.3447, 2.7895, -0.5585, 1.6562, -3.4243, -0.9751,0.6078, -0.6654, -1.5980, 0.0568, 1.0073, -4.0373, -1.1408, 1.3027,-0.0781, 2.2652, -2.5808, 0.5551, 1.7056, 0.6155, 0.3708, -0.7449,0.7294, -1.6789, 0.2668, 1.3637, -1.1435, -4.5174, 0.1851, -0.4093,-0.1503, 0.4865, -0.7953, -1.6489, -0.5183, 0.6161, -0.5087, -1.3621,3.3161, 0.3884, -1.0508, 0.5203, 0.2696, -1.4678, -1.4626, 0.9397,-7.0490, -0.6900, 3.3881, -0.6778, -1.4596, 0.1268, 8.7628, -1.0302,1.3928, -0.4755, -0.1050, -1.2061 ])
stardist = stats.norm(loc=np.mean(x_i),scale=np.std(x_i, ddof=1))
sourcedist = stats.norm

lilliefors(x_i, stardist, sourcedist)

(0.11334112306628319, 0.00307)

## Cramér-von Mises

...

In [25]:
def cramer_von_mises(x_i, stardist, sourcedist):
    x_i.sort()
    n = len(x_i)
    
    Fstar_i = stardist.cdf(x_i)
    Fhatp_i = np.arange(n)/n 
    Fhatm_i = (1+np.arange(n))/n
    
    i_i = np.arange(1,n+1) 
    TCvM = 1./(12.*n) + np.sum(((2*i_i-1)/(2.*n)-Fstar_i)**2)
    
    np.random.seed(20230327)
    Nmonte = 10**5
    x_Ii = sourcedist.rvs(size=(Nmonte,n))
    x_Ii.sort(axis=-1)
    
    xbar_I = np.mean(x_Ii,axis=-1)
    s_I = np.std(x_Ii,axis=-1,ddof=1)
    
    Fstar_Ii = sourcedist.cdf((x_Ii-xbar_I[:,None])/s_I[:,None])
    
    TCvM_I = 1./(12.*n) + np.sum(((2*i_i[None,:]-1)/(2.*n)-Fstar_Ii)**2,axis=-1)
    
    p = np.mean(TCvM_I>=TCvM)
    
    return TCvM, p

In [26]:
cramer_von_mises(x_i, stardist, sourcedist)

(0.3373679828059199, 6e-05)

## Anderson-Darling

...

In [27]:
def anderson_darling(x_i, stardist, sourcedist):
    x_i.sort()
    n = len(x_i)
    
    Fstar_i = stardist.cdf(x_i)
    Fhatp_i = np.arange(n)/n 
    Fhatm_i = (1+np.arange(n))/n
    
    i_i = np.arange(1,n+1) 
    A2 = -n-np.sum(((2*i_i-1.)/n)*(np.log(Fstar_i*(1-Fstar_i[::-1]))))
    
    np.random.seed(20230327)
    Nmonte = 10**5
    x_Ii = sourcedist.rvs(size=(Nmonte,n))
    x_Ii.sort(axis=-1)
    
    xbar_I = np.mean(x_Ii,axis=-1)
    s_I = np.std(x_Ii,axis=-1,ddof=1)
    
    Fstar_Ii = sourcedist.cdf((x_Ii-xbar_I[:,None])/s_I[:,None])
    
    A2_I = -n-np.sum(((2*i_i[None,:]-1.)/n)*(np.log(Fstar_Ii*(1-Fstar_Ii[:,::-1]))),axis=-1)
    
    p = np.mean(A2_I>=A2), np.max(A2_I)
    
    return A2, p

In [28]:
anderson_darling(x_i, stardist, sourcedist)

(2.0771241595155914, (0.0, 2.0148593650561395))

## Two-Sample Kolmogorov-Smirnov

...

In [29]:
def two_sample_kolmogorov(x_i, y_j):
    n = len(x_i)
    m = len(y_j)
    N=n+m
    x_i.sort()
    y_j.sort()
    
    Fxhat_i = (1.+np.arange(n))/n
    Fyhat_j = (1.+np.arange(m))/m
    X_k = np.concatenate((x_i,y_j))
    X_k.sort()
    
    Fxhat_k = np.mean(x_i[None,:] <= X_k[:,None], axis=-1)
    Fyhat_k = np.mean(y_j[None,:] <= X_k[:,None], axis=-1)

    Tp = max(Fxhat_k-Fyhat_k)
    Tm = max(Fyhat_k-Fxhat_k)
    T = max(Tp,Tm)
    
    Rxy_k = stats.rankdata(np.concatenate((x_i,y_j))) 
    Rx_i=Rxy_k[:n] 
    Ry_j=Rxy_k[n:]
    RX_k = np.sort(Rxy_k)
    
    xranks_Ii = np.array([xranks_i for xranks_i in itertools.combinations(RX_k,n)])
    yranks_Ij = np.array([np.setdiff1d(RX_k,xranks_i) for xranks_i in xranks_Ii])
    
    Fxhat_Ik = np.mean(xranks_Ii[:,None,:]<=RX_k[None,:,None],axis=-1)
    Fyhat_Ik = np.array([np.mean(yranks_j[None,:]<=RX_k[:,None], axis=-1) for yranks_j in yranks_Ij])
    
    Tp_I = np.max(Fxhat_Ik-Fyhat_Ik,axis=-1)
    Tm_I = np.max(Fyhat_Ik-Fxhat_Ik,axis=-1)
    T_I = np.max(np.abs(Fxhat_Ik-Fyhat_Ik),axis=-1)
    
    Tp_I = np.round(Tp_I,decimals=8)
    Tm_I = np.round(Tm_I,decimals=8)
    T_I = np.maximum(Tp_I,Tm_I); np.unique(T_I)
    
    p = np.mean(T_I>=T)
    
    return T, p

In [30]:
x_i = np.array([3.14,0.1,2.72]) 
y_j = np.array([2,4])
two_sample_kolmogorov(x_i, y_j)

(0.5, 0.9)

## Two-Sample Cramér-von Mises 

...

In [33]:
def two_sample_cramer_von(x_i, y_j):
    n = len(x_i)
    m = len(y_j)
    N=n+m
    x_i.sort()
    y_j.sort()
    
    Fxhat_i = (1.+np.arange(n))/n
    Fyhat_j = (1.+np.arange(m))/m
    X_k = np.concatenate((x_i,y_j))
    X_k.sort()
    
    Fxhat_k = np.mean(x_i[None,:] <= X_k[:,None], axis=-1)
    Fyhat_k = np.mean(y_j[None,:] <= X_k[:,None], axis=-1)

    TCvM = n*m/N**2 * np.sum((Fxhat_k-Fyhat_k)**2)
    TCvM = np.round(TCvM,8)
    
    Rxy_k = stats.rankdata(np.concatenate((x_i,y_j))) 
    Rx_i=Rxy_k[:n] 
    Ry_j=Rxy_k[n:]
    RX_k = np.sort(Rxy_k)
    
    xranks_Ii = np.array([xranks_i for xranks_i in itertools.combinations(RX_k,n)])
    yranks_Ij = np.array([np.setdiff1d(RX_k,xranks_i) for xranks_i in xranks_Ii])
    
    Fxhat_Ik = np.mean(xranks_Ii[:,None,:]<=RX_k[None,:,None],axis=-1)
    Fyhat_Ik = np.array([np.mean(yranks_j[None,:]<=RX_k[:,None], axis=-1) for yranks_j in yranks_Ij])
    
    TCvM_I = n*m/N**2 * np.sum((Fxhat_Ik-Fyhat_Ik)**2,axis=-1)
    TCvM_I = np.round(TCvM_I,8)
    
    p = np.mean(TCvM_I >= TCvM)
    
    return TCvM, p

In [34]:
two_sample_cramer_von(x_i, y_j)

(0.1, 0.9)

## Two-Sample Anderson Darling 

...

In [36]:
def two_sample_anderson(x_i, y_j):
    n = len(x_i)
    m = len(y_j)
    N=n+m
    x_i.sort()
    y_j.sort()
    k_k = 1 + np.arange(N)

    Fxhat_i = (1.+np.arange(n))/n
    Fyhat_j = (1.+np.arange(m))/m
    X_k = np.concatenate((x_i,y_j))
    X_k.sort()
    
    Fxhat_k = np.mean(x_i[None,:] <= X_k[:,None], axis=-1)
    Fyhat_k = np.mean(y_j[None,:] <= X_k[:,None], axis=-1)
    
    A2 = n*m*np.sum( (Fxhat_k[:-1]-Fyhat_k[:-1])**2 / (k_k[:-1]*(N-k_k[:-1])) )
    A2 = np.round(A2,8)
    
    Rxy_k = stats.rankdata(np.concatenate((x_i,y_j))) 
    Rx_i=Rxy_k[:n] 
    Ry_j=Rxy_k[n:]
    RX_k = np.sort(Rxy_k)
    
    xranks_Ii = np.array([xranks_i for xranks_i in itertools.combinations(RX_k,n)])
    yranks_Ij = np.array([np.setdiff1d(RX_k,xranks_i) for xranks_i in xranks_Ii])
    
    Fxhat_Ik = np.mean(xranks_Ii[:,None,:]<=RX_k[None,:,None],axis=-1)
    Fyhat_Ik = np.array([np.mean(yranks_j[None,:]<=RX_k[:,None], axis=-1) for yranks_j in yranks_Ij])
        
    A2_I = n*m*np.sum( (Fxhat_Ik[:,:-1]-Fyhat_Ik[:,:-1])**2 / (k_k[None,:-1]*(N-k_k[None,:-1])), axis=-1 )
    A2_I = np.round(A2_I,8)
    
    p = np.mean(A2_I>=A2)
    
    return A2, p

In [38]:
two_sample_anderson(x_i, y_j)

(0.59722222, 0.9)